In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import gc
print(datetime.datetime.now())

2020-07-13 15:31:28.636329


In [2]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [3]:
list_pos_q1=list(recursive_file_gen("/home/jian/BigLots/2020_by_weeks/"))
list_pos_q1=[x for x in list_pos_q1 if "dailysales" in x.lower()]
list_pos_q1=[x for x in list_pos_q1 if x.split("_weeks/MediaStorm_")[1][:10]>="2020-02-02"]
list_pos_q1=[x for x in list_pos_q1 if x.split("_weeks/MediaStorm_")[1][:10]<="2020-05-02"]
len(list_pos_q1)


13

In [4]:
df_CN=pd.read_csv("./notexposed_control_idlist.csv",usecols=["customer_id_hashed"])
df_TE=pd.read_csv("./exposed_test_idlist.csv",usecols=["customer_id_hashed"])
df_CN['group']="CN"
df_TE['group']="TE"

df_id_list=df_CN.append(df_TE)
del df_CN
del df_TE
print(df_id_list.shape,df_id_list['customer_id_hashed'].nunique())
df_count_by_group=df_id_list.groupby('group')['customer_id_hashed'].nunique().to_frame().reset_index()
gc.collect()

(1010014, 2) 1010014


0

In [5]:
585058+424956

1010014

In [6]:
agg_func={'customer_id_hashed':"nunique",'item_transaction_amt':"sum"}
df_output=pd.DataFrame()
for file in list_pos_q1:
    df=pd.read_table(file,sep="|")
    df=pd.merge(df_id_list,df,on="customer_id_hashed",how="inner")
    df_sales=df.groupby(['group'])['customer_id_hashed','item_transaction_amt'].agg(agg_func).reset_index().rename(columns={"customer_id_hashed":"unique_shopper","item_transaction_amt":"sales"})
    df_trans=df[['customer_id_hashed','group','location_id','transaction_dt','transaction_id']].drop_duplicates()
    df_trans=df_trans.groupby(['group'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"transactions"})
    df=pd.merge(df_sales,df_trans,on="group")
    df.insert(0,'week_end_dt',file.split("_weeks/MediaStorm_")[1][:10])
    df_output=df_output.append(df)
    print(file,datetime.datetime.now())

/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-02-08/MediaStormDailySales20200211-120911-483.txt 2020-07-13 15:31:59.272832
/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-02-15/MediaStormDailySales20200218-110927-085.txt 2020-07-13 15:32:26.773553
/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-02-22/MediaStormDailySales20200225-111349-615.txt 2020-07-13 15:32:55.152439
/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-02-29/MediaStormDailySales20200303-111359-260.txt 2020-07-13 15:33:26.435741
/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-03-07/MediaStormDailySales20200311-125005-860.txt 2020-07-13 15:34:01.515078
/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-03-28/MediaStormDailySales20200331-112303-873.txt 2020-07-13 15:34:28.942117
/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-03-14/MediaStormDailySales20200317-112349-402.txt 2020-07-13 15:35:12.719228
/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-03-21/MediaStormDailySales20200324-112302-496.txt 2020-07-13 15:35:50

In [11]:
writer=pd.ExcelWriter("./BL_Q1_1to1_by_week_JL_%s.xlsx"%str(datetime.datetime.now().date()),engine="xlsxwriter")
df_output.to_excel(writer,"by_week",index=False)
df_count_by_group.to_excel(writer,"total_ids",index=False)
writer.save()